In [1]:
import pandas as pd
from pathlib import Path
from src.utils import get_project_root
from src.categorization_helpers import *
from src.swinno_helpers import *

ROOT = get_project_root()

engine = connect_swinno_db()


In [2]:
article_check = pd.read_excel(Path(ROOT, "data/raw-data/check_articles.xlsx"))
article_check.dropna(subset="article_checked", inplace=True)
article_check.head()
print(article_check.shape)


(177, 9)


In [3]:
innovation_types = article_check.loc[:, ["sinno_id", "innovation_type"]]
cleaned_innovation_types = clean_codes(
    innovation_types, code_digits=3, column="innovation_type"
)

visions = article_check.loc[:, ["sinno_id", "bioeconomy_vision"]]
cleaned_visions = clean_codes(visions, code_digits=1, column="bioeconomy_vision")
notes = article_check.loc[:, ["sinno_id", "notes"]]

cleaned_innovation_types.name = "innovation_types"
cleaned_visions.name = "visions"
notes.name = "notes"


In [4]:
for df in [cleaned_innovation_types, notes, cleaned_visions]:
    csv_path = Path(ROOT, "data", "modified-data", f"{df.name}-duplicates.csv")
    check_duplicates(df, output_name=df.name, output_path=csv_path)
    print("-" * 5)


split_visions = split_cols(cleaned_visions, col_to_split="bioeconomy_vision", sep=",")
melted_visions = melt_table(
    split_visions, id_vars="sinno_id", col_start="bio", value_name="bioeconomy_vision"
)

melted_visions = melted_visions.dropna()

melted_visions.to_sql(
    name="bioeconomy_visions_articles", con=engine, if_exists="replace", index=False
)

split_innovation_types = split_cols(
    cleaned_innovation_types, col_to_split="innovation_type", sep=","
)
melted_innovation_types = melt_table(
    split_innovation_types,
    id_vars="sinno_id",
    col_start="innovation",
    value_name="innovation_type",
)
melted_innovation_types.dropna()
melted_innovation_types.to_sql(
    name="eco_innovations_articles", con=engine, if_exists="replace", index=False
)

notes = notes.dropna()
notes.to_sql(
    name="categorization_notes_articles", con=engine, index=False, if_exists="replace"
)


No duplicates found.
-----
No duplicates found.
-----
No duplicates found.
-----


64